## Advanced Lane Finding Project

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

In [1]:
# Importing libraries
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pickle
import os
%matplotlib qt

# Camera Calibration
Reference from: [here](https://github.com/udacity/CarND-Camera-Calibration/blob/master/camera_calibration.ipynb)

In [2]:
def camera_calibration(nx = 6,ny = 9):
    objp = np.zeros((nx * ny,3), np.float32)
    objp[:,:2] = np.mgrid[0:ny, 0:nx].T.reshape(-1,2)
    objpoints = [] # 3d points in real world space
    imgpoints = [] # 2d points in image plane.
    images = glob.glob('camera_cal/calibration*.jpg')
    for idx, fname in enumerate(images):
        img = cv2.imread(fname)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # Find the chessboard corners
        ret, corners = cv2.findChessboardCorners(gray, (ny,nx), None)

        # If found, add object points, image points
        if ret == True:
            objpoints.append(objp)
            imgpoints.append(corners)
    # Test undistortion on an image
    img = cv2.imread('camera_cal/calibration1.jpg')
    img_size = (img.shape[1], img.shape[0])

    # Do camera calibration given object points and image points
    retval, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img_size,None,None)

    dst = cv2.undistort(img, mtx, dist, None, mtx)
    vis = np.concatenate((img, dst), axis=1)
    cv2.imwrite('output_images/calibration1_undist.jpg',vis)
    
    # Save the camera calibration result
    dist_pickle = {}
    dist_pickle["mtx"] = mtx
    dist_pickle["dist"] = dist
    outfile = open("cam_cal.p", "wb")
    pickle.dump(dist_pickle, outfile)
    outfile.close()
    return {"mtx":mtx, "dist":dist}

In [3]:
def cal_test(dist_pickle):
    images = glob.glob('test_images/*.jpg') 
    for path in images:
        img = cv2.imread(path)
        _, filename = os.path.split(path)
        dst = cv2.undistort(img, dist_pickle["mtx"], dist_pickle["dist"], None, dist_pickle["mtx"])
        output_path = 'output_images/cal_test/cal_' + filename
        vis = np.concatenate((img, dst), axis=1)
        cv2.imwrite(output_path,vis)

# Gradient Filtering 

In [45]:
def abs_sobel_thresh(gray, axis, sobel_kernel=3, thresh=(20, 100)):
    if axis == 'x':
        sobel = cv2.Sobel(gray, cv2.CV_64F, 1, 0)
    else:
        sobel = cv2.Sobel(gray, cv2.CV_64F, 0, 1)
        
    abs_sobel = np.absolute(sobel)
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))
    
    grad_binary = np.zeros_like(gray)
    grad_binary[((scaled_sobel > thresh[0]) & (scaled_sobel < thresh[1]))] = 255
    return grad_binary

def mag_thresh(gray, sobel_kernel=3, mag_thresh=(0, 255)):
    # Calculate gradient magnitude
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1)
    abs_mag = np.sqrt(sobelx**2 + sobely**2)
    scaled = np.uint8(255*abs_mag/np.max(abs_mag))
    # Apply threshold
    mag_binary = np.zeros_like(gray)
    mag_binary[((scaled > mag_thresh[0])&(scaled < mag_thresh[1]))] = 255
    return mag_binary

def dir_threshold(gray, sobel_kernel=3, thresh=(0, np.pi/2)):
    # Calculate gradient direction
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1)
    abs_x = np.absolute(sobelx)
    abs_y = np.absolute(sobely)
    direct = np.arctan2(abs_y, abs_x)
    binary_output = np.zeros_like(gray)
    binary_output[(direct > thresh[0]) & (direct < thresh[1])] = 255
    return binary_output

def filtering(ksize = 3, path='test_images/straight_lines1.jpg', img = None):
    if len(img) == 0:
        img = mpimg.imread(path)
    gray_image = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    blur_gray = cv2.GaussianBlur(gray_image,(3,3),0)
    # Apply each of the thresholding functions
    gradx = abs_sobel_thresh(gray_image, axis = 'x',sobel_kernel=ksize, thresh=(25, 200))
    dir_binary = dir_threshold(blur_gray, sobel_kernel=ksize, thresh=(np.pi/4, np.pi/4 + 0.3))
    gradx_direct = np.zeros_like(gray_image)
    gradx_direct[(gradx == 255)&(dir_binary == 255)] = 255
    return gradx, dir_binary, gradx_direct

def filtering_test():
    images = glob.glob('test_images/*.jpg') 
    for path in images:
        img = cv2.imread(path)
        _, filename = os.path.split(path)
        gradx, dir_binary, gradx_dir = filtering(path = path)
        output_path_x = 'output_images/filter_test/gradx_' + filename
        output_path_dir = 'output_images/filter_test/dir_' + filename
        output_path_x_dir = 'output_images/filter_test/x_dir_' + filename
        cv2.imwrite(output_path_x,gradx)
        cv2.imwrite(output_path_dir,dir_binary)
        cv2.imwrite(output_path_x_dir,dir_binary)


In [46]:
cv2.destroyAllWindows()

# Color Filtering

In [47]:
def color_filtering(path='test_images/straight_lines1.jpg', img = None, thresh = (200,255)):
    if len(img) == 0:
        img = mpimg.imread(path)
    _, filename = os.path.split(path)
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    S = hls[:,:,2]
    binary = np.zeros_like(S)
    binary[(S > thresh[0]) & (S <= thresh[1])] = 255
    return binary
#     cv2.imwrite("output_images/color_test/" + filename, binary)
def color_test():
    images = glob.glob('test_images/*.jpg') 
    for path in images:
        img = cv2.imread(path)
        _, filename = os.path.split(path)
        S = color_filtering(path)
        output_path = 'output_images/color_test/' + filename
        cv2.imwrite(output_path,S)


In [75]:
def pipeline_test():
    images = glob.glob('test_images/*.jpg') 
    #read in cal file or re-cal
    dist_pickle = {}
    try:
        infile = open("cam_cal.p",'rb')
        dist_pickle = pickle.load(infile)
    except:
        dist_pickle = camera_calibration()
    for path in images:
        img = cv2.imread(path)
        _, filename = os.path.split(path)
        
        # undistortion
        img = cv2.undistort(img, dist_pickle["mtx"], dist_pickle["dist"], None, dist_pickle["mtx"])
        
        # gradient filtering
        gradx, dir_binary, gradx_direct = filtering(img = img, path = path)
        
        # color filtering
        S = color_filtering(img = img, path = path)
        
        # stack
        color = np.dstack(( np.zeros_like(gradx_direct), gradx_direct, S))
        
        #Combine
        combined_binary = np.zeros_like(gradx_direct)
        combined_binary[(S == 255) | (gradx_direct == 255)] = 255
        combined_binary = np.dstack((combined_binary,combined_binary,combined_binary))
        
        # output
        output_path_s = 'output_images/pipeline_test/s_' + filename
        output_path_x = 'output_images/pipeline_test/x_' + filename
        output_path_dir = 'output_images/pipeline_test/dir_' + filename
        output_path_x_dir = 'output_images/pipeline_test/x_dir_' + filename
        output_path_origin = 'output_images/pipeline_test/origin_' + filename
        output_path_color = 'output_images/pipeline_test/c_' + filename
        output_path_combined = 'output_images/pipeline_test/combined_' + filename
        vis = np.concatenate((img, combined_binary), axis=1)
        
#         cv2.imwrite(output_path_s,S)
#         cv2.imwrite(output_path_x,gradx)
#         cv2.imwrite(output_path_dir,dir_binary)
#         cv2.imwrite(output_path_x_dir,gradx_direct)
#         cv2.imwrite(output_path_origin,img)
#         cv2.imwrite(output_path_color,color)
#         cv2.imwrite(output_path_comined,combined_binary)
        cv2.imwrite(output_path_combined,vis)
        

In [76]:
pipeline_test()